# Introduction
This notebook shows how to connect to a crypto exchange and receive a trade feed from it. We then start the run using this trade feed. This notebook use the XChange feed, which based on the nice XChange Java library that contains connectivity to many of crypto exchanges.

Besides this generic feed, roboquant also has dedicated support for binance. For that please check binance notebook.

In [ ]:
%use roboquant
Welcome()

# Extra libraries
Roboquant includes the base classes for crypto exchange feeds and trading. So if we want to connect to real exchanges, we'll need to

1. Load the exchange specific libraries, like bitstamp in this example.
2. Import the packages for the exchange we want to connect to.

Check the XChange website for more details on all the available exchanges. In this example we use Bitstamp

In [ ]:
@file:DependsOn("org.knowm.xchange:xchange-bitstamp:5.0.6")
@file:DependsOn("org.knowm.xchange:xchange-stream-bitstamp:5.0.6")

import org.roboquant.xchange.*
import info.bitrich.xchangestream.bitstamp.v2.BitstampStreamingExchange
import info.bitrich.xchangestream.core.StreamingExchangeFactory

# Setup
We now create, as usual, the strategy we want to test. There is nothing specific to crypto trading that needs to be taken care of.

In [ ]:
 val strategy = TAStrategy(12)

strategy.buy { 
    (ta.cdlMorningStar(it) || ta.cdlMorningDojiStar(it)) && ta.ema(it.close, 5) > ta.ema(it.close, 9)
}

strategy.sell { 
    ta.cdl2Crows(it) && ta.ema(it.close, 3) < ta.ema(it.close, 7)
}


# Feed
Now we create an instance of the Bitstamp exchange and connect to it. Then we are fine to use it as a feed and subscribe to one or more currency pairs. We subscribe to ticker info since it contains the OHLC pricing info we need for our strategy.

In [ ]:
val exchange = StreamingExchangeFactory.INSTANCE.createExchange(BitstampStreamingExchange::class.java)
exchange.connect().blockingAwait()

val feed = XChangeFeed(exchange)
feed.subscribeTicker(Pair("BTC", "USD"))

In [ ]:
val roboquant =  Roboquant(strategy, AccountSummary(), ProgressMetric())

# Live Test
All that remains, is to start run to roboquant and evaluate the strategy against the feed. We'll run it for 10 minutes. If the time displayed looks funny, don't forget that roboquant uses a timezone independent representation for all internal time processing and not your local timezone.

In [ ]:
val timeFrame = TimeFrame.nextMinutes(10)
roboquant.run(feed, timeFrame)

# Disconnect
And we disconnect from the exchnage

In [ ]:
exchange.disconnect().blockingAwait()

In [ ]:
roboquant.broker.account.summary()

In [ ]:
roboquant.logger.summary()